# Importing Dependencies

In [1]:
#Importing Library
import pandas as pd
import numpy as np
import re
import pickle

#Importing Natural Language Toolkit Libraries
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

#Importing Scikit-Learn Libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score


C:\Users\Swaroop Honrao\Python\Python370\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


# Load Dataset

In [2]:
data = pd.read_csv('IMDB-Dataset.csv')
print(data.shape)
data.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [4]:
data.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

# Data Pre-Processing

In [5]:
#Replacing positive and Negative sentiments to 1 and 0 respectively.
data.sentiment.replace('positive', 1, inplace=True)
data.sentiment.replace('negative', 0, inplace=True)
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
5,"Probably my all-time favorite movie, a story o...",1
6,I sure would like to see a resurrection of a u...,1
7,"This show was an amazing, fresh & innovative i...",0
8,Encouraged by the positive comments about this...,0
9,If you like original gut wrenching laughter yo...,1


In [6]:
data.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

# Review Cleaning

# 1. Remove HTML tags (‘<.*?>’)

In [7]:
def clean(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned, '', text)

data.review = data.review.apply(clean)
data.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

# 2. Remove Special Characters.

In [9]:
def is_special(text):
    rem = ''
    for i in text:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
    return rem

data.review = data.review.apply(is_special)
data.review[0]

'Oneoftheotherreviewershasmentionedthatafterwatchingjust1OzepisodeyoullbehookedTheyarerightasthisisexactlywhathappenedwithmeThefirstthingthatstruckmeaboutOzwasitsbrutalityandunflinchingscenesofviolencewhichsetinrightfromthewordGOTrustmethisisnotashowforthefaintheartedortimidThisshowpullsnopuncheswithregardstodrugssexorviolenceItsishardcoreintheclassicuseofthewordItiscalledOZasthatisthenicknamegiventotheOswaldMaximumSecurityStatePenitentaryItfocusesmainlyonEmeraldCityanexperimentalsectionoftheprisonwhereallthecellshaveglassfrontsandfaceinwardssoprivacyisnothighontheagendaEmCityishometomanyAryansMuslimsgangstasLatinosChristiansItaliansIrishandmoresoscufflesdeathstaresdodgydealingsandshadyagreementsareneverfarawayIwouldsaythemainappealoftheshowisduetothefactthatitgoeswhereothershowswouldntdareForgetprettypicturespaintedformainstreamaudiencesforgetcharmforgetromanceOZdoesntmessaroundThefirstepisodeIeversawstruckmeassonastyitwassurrealIcouldntsayIwasreadyforitbutasIwatchedmoreIdevelopedatas

# 3. Convert to lower Case

In [10]:
def to_lower(text):
    return text.lower()

data.review = data.review.apply(to_lower)
data.review[0]

'oneoftheotherreviewershasmentionedthatafterwatchingjust1ozepisodeyoullbehookedtheyarerightasthisisexactlywhathappenedwithmethefirstthingthatstruckmeaboutozwasitsbrutalityandunflinchingscenesofviolencewhichsetinrightfromthewordgotrustmethisisnotashowforthefaintheartedortimidthisshowpullsnopuncheswithregardstodrugssexorviolenceitsishardcoreintheclassicuseoftheworditiscalledozasthatisthenicknamegiventotheoswaldmaximumsecuritystatepenitentaryitfocusesmainlyonemeraldcityanexperimentalsectionoftheprisonwhereallthecellshaveglassfrontsandfaceinwardssoprivacyisnothighontheagendaemcityishometomanyaryansmuslimsgangstaslatinoschristiansitaliansirishandmoresoscufflesdeathstaresdodgydealingsandshadyagreementsareneverfarawayiwouldsaythemainappealoftheshowisduetothefactthatitgoeswhereothershowswouldntdareforgetprettypicturespaintedformainstreamaudiencesforgetcharmforgetromanceozdoesntmessaroundthefirstepisodeieversawstruckmeassonastyitwassurrealicouldntsayiwasreadyforitbutasiwatchedmoreidevelopedatas

# 4. Remove Stopwoards

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Swaroop
[nltk_data]     Honrao\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [13]:
def rem_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    return [w for w in words if w not in stop_words]

data.review = data.review.apply(rem_stopwords)
data.review[0]

['oneoftheotherreviewershasmentionedthatafterwatchingjust1ozepisodeyoullbehookedtheyarerightasthisisexactlywhathappenedwithmethefirstthingthatstruckmeaboutozwasitsbrutalityandunflinchingscenesofviolencewhichsetinrightfromthewordgotrustmethisisnotashowforthefaintheartedortimidthisshowpullsnopuncheswithregardstodrugssexorviolenceitsishardcoreintheclassicuseoftheworditiscalledozasthatisthenicknamegiventotheoswaldmaximumsecuritystatepenitentaryitfocusesmainlyonemeraldcityanexperimentalsectionoftheprisonwhereallthecellshaveglassfrontsandfaceinwardssoprivacyisnothighontheagendaemcityishometomanyaryansmuslimsgangstaslatinoschristiansitaliansirishandmoresoscufflesdeathstaresdodgydealingsandshadyagreementsareneverfarawayiwouldsaythemainappealoftheshowisduetothefactthatitgoeswhereothershowswouldntdareforgetprettypicturespaintedformainstreamaudiencesforgetcharmforgetromanceozdoesntmessaroundthefirstepisodeieversawstruckmeassonastyitwassurrealicouldntsayiwasreadyforitbutasiwatchedmoreidevelopedata

# 5. Stem the Words

In [14]:
def stem_text(text):
    ss = SnowballStemmer('english')
    return " ".join([ss.stem(w) for w in text])

data.review = data.review.apply(stem_text)
data.review[0]

'oneoftheotherreviewershasmentionedthatafterwatchingjust1ozepisodeyoullbehookedtheyarerightasthisisexactlywhathappenedwithmethefirstthingthatstruckmeaboutozwasitsbrutalityandunflinchingscenesofviolencewhichsetinrightfromthewordgotrustmethisisnotashowforthefaintheartedortimidthisshowpullsnopuncheswithregardstodrugssexorviolenceitsishardcoreintheclassicuseoftheworditiscalledozasthatisthenicknamegiventotheoswaldmaximumsecuritystatepenitentaryitfocusesmainlyonemeraldcityanexperimentalsectionoftheprisonwhereallthecellshaveglassfrontsandfaceinwardssoprivacyisnothighontheagendaemcityishometomanyaryansmuslimsgangstaslatinoschristiansitaliansirishandmoresoscufflesdeathstaresdodgydealingsandshadyagreementsareneverfarawayiwouldsaythemainappealoftheshowisduetothefactthatitgoeswhereothershowswouldntdareforgetprettypicturespaintedformainstreamaudiencesforgetcharmforgetromanceozdoesntmessaroundthefirstepisodeieversawstruckmeassonastyitwassurrealicouldntsayiwasreadyforitbutasiwatchedmoreidevelopedatas

In [15]:
data.head()

,review,sentiment
0,oneoftheotherreviewershasmentionedthatafterwat...,1
1,awonderfullittleproductionthefilmingtechniquei...,1
2,ithoughtthiswasawonderfulwaytospendtimeonatooh...,1
3,basicallytheresafamilywherealittleboyjakethink...,0
4,pettermatteisloveinthetimeofmoneyisavisuallyst...,1


# Creating The Model

# 1. Creating Bag of Words

In [16]:
X = np.array(data.iloc[:, 0].values)
y = np.array(data.sentiment.values)
cv = CountVectorizer(max_features=1000)
X = cv.fit_transform(data.review).toarray()

print("Shape of X is: ", X.shape)
print("Shape of y is: ", y.shape)

Shape of X is:  (50000, 1000)
Shape of y is:  (50000,)


In [17]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [18]:
y

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

# Splitting Dataset into Training and Test Set

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)
print("Training Data Set Shape is: X = {}, y ={}".format(X_train.shape, y_train.shape))
print("Test Data Set Shape is: X = {}, y ={}".format(X_test.shape, y_test.shape))

Training Data Set Shape is: X = (40000, 1000), y =(40000,)
Test Data Set Shape is: X = (10000, 1000), y =(10000,)


# Train Model using Navie Bayes

In [29]:
gnb = GaussianNB()
mnb = MultinomialNB(alpha=1.0, fit_prior=True)
bnb = BernoulliNB(alpha=1.0, fit_prior=True)

#Assigning Train set to the Models
gnb.fit(X_train, y_train)
mnb.fit(X_train, y_train)
bnb.fit(X_train, y_train)

BernoulliNB()

# Prediction and Model Accuracy

In [30]:
ypred_g = gnb.predict(X_test)
ypred_m = mnb.predict(X_test)
ypred_b = bnb.predict(X_test)

print("Gaussian: ", accuracy_score(y_test, ypred_g))
print("Multinominal: ", accuracy_score(y_test, ypred_m))
print("Bernoulli: ", accuracy_score(y_test, ypred_b))

Gaussian:  0.5066
Multinominal:  0.5066
Bernoulli:  0.5066


In [31]:
pickle.dump(bnb,open('model1.pkl','wb'))

In [33]:
rev =  """Terrible. Complete trash. Brainless tripe. Insulting to anyone who isn't an 8 year old fan boy. Im actually pretty disgusted that this movie is making the money it is - what does it say about the people who brainlessly hand over the hard earned cash to be 'entertained' in this fashion and then come here to leave a positive 8.8 review?? Oh yes, they are morons. Its the only sensible conclusion to draw. How anyone can rate this movie amongst the pantheon of great titles is beyond me.

So trying to find something constructive to say about this title is hard...I enjoyed Iron Man? Tony Stark is an inspirational character in his own movies but here he is a pale shadow of that...About the only 'hook' this movie had into me was wondering when and if Iron Man would knock Captain America out...Oh how I wished he had :( What were these other characters anyways? Useless, bickering idiots who really couldn't organise happy times in a brewery. The film was a chaotic mish mash of action elements and failed 'set pieces'...

I found the villain to be quite amusing. And now I give up. This movie is not robbing any more of my time but I felt I ought to contribute to restoring the obvious fake rating and reviews this movie has been getting on IMDb."""

f1 = clean(rev)
f2 = is_special(f1)
f3 = to_lower(f2)
f4 = rem_stopwords(f3)
f5 = stem_text(f4)

bow,words = [],word_tokenize(f5)
for word in words:
    bow.append(words.count(word))
#np.array(bow).reshape(1,3000)
#bow.shape
word_dict = cv.vocabulary_
pickle.dump(word_dict,open('bow.pkl','wb'))

In [34]:
inp = []
for i in word_dict:
    inp.append(f5.count(i[0]))
y_pred = bnb.predict(np.array(inp).reshape(1,1000))

In [35]:
y_pred

array([0], dtype=int64)